In [10]:
import pandas as pd
from tqdm.auto import tqdm
import random
from rdkit import Chem
from bson.binary import Binary
import pickle
import numpy as np
import pyarrow.parquet as pq
from collections import defaultdict
from simple_gnn.preprocess import create_atoms, create_ijbonddict, extract_fingerprints

In [8]:
# Define dictionaries
atom_dict = {'C': 0, 'N': 1, ('C', 'aromatic'): 2, ('N', 'aromatic'): 3, 'O': 4, 'H': 5, ('S', 'aromatic'): 6, 'Cl': 7, 'S': 8, ('O', 'aromatic'): 9, 'Br': 10, 'F': 11, 'Si': 12, 'B': 13, 'I': 14}
bond_dict = {'TRIPLE': 0, 'SINGLE': 1, 'AROMATIC': 2, 'DOUBLE': 3}
fingerprint_dict = {i: i for i in range(15)}
edge_dict = {}


In [1]:
from pymongo import MongoClient
def get_database():
 
   # Provide the mongodb atlas url to connect python to mongodb using pymongo
   CONNECTION_STRING = "mongodb://anonymous-server.local/belka"
 
   # Create a connection using MongoClient. You can import MongoClient or use pymongo.MongoClient
   client = MongoClient(CONNECTION_STRING)
 
   # Create the database for our example (we will use the same database throughout the tutorial
   return client['belka']
  
   # Get the database
dbname = get_database()
collection_name = dbname["full_molecules_fingerprints"]

In [2]:
from pymongo import UpdateOne
from tqdm.auto import tqdm
# Retrieve all documents, optionally sorting them if order matters
documents = collection_name.find().sort('_id', 1)

# Prepare bulk operations
bulk_operations = []
next_id = 1

for document in tqdm(documents):
    # Prepare the update operation with a new sequential `id`
    operation = UpdateOne({'_id': document['_id']}, {'$set': {'seq_id': next_id}})
    bulk_operations.append(operation)
    next_id += 1

    # Execute the batch update every 1000 items or choose your batch size
    if len(bulk_operations) >= 1000:
        collection_name.bulk_write(bulk_operations)
        bulk_operations = []  # Reset the list of operations after executing

# Execute any remaining operations in the last batch
if bulk_operations:
    collection_name.bulk_write(bulk_operations)

/home/anonymous/miniconda3/envs/belka/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
0it [00:00, ?it/s]

33601111it [45:18, 12358.36it/s]


In [2]:
unqiue_test_ids = list(collection_name.find({},{'_id':0,'id':1}))

In [4]:
import pickle
with open('unique_test_ids.pkl','wb') as f:
    pickle.dump([x['id'] for x in unqiue_test_ids],f)

In [22]:
parquet_file = pq.ParquetFile('test.parquet')
for batch in tqdm(parquet_file.iter_batches(batch_size=1000)):
    batch_df = batch.to_pandas()
    mol = [ Chem.AddHs(Chem.MolFromSmiles(x.replace("Dy","C"))) for x in batch_df['molecule_smiles']]
    atoms = [create_atoms(x, atom_dict) for x in mol ]
    molecular_size = [len(x) for x in atoms]
    i_jbond_dict = [create_ijbonddict(x, bond_dict) for x in mol]
    fingerprints = [extract_fingerprints(0, x, y, fingerprint_dict, edge_dict) for (x,y) in zip(atoms, i_jbond_dict)]
    adjacency = [Chem.GetAdjacencyMatrix(x) for x in mol]
    batch_df['fingerprints'] = [x.tolist() for x in fingerprints]
    batch_df['adjacency'] = [Binary(pickle.dumps(x.astype(np.uint8), protocol=2), subtype=128 ) for x in fingerprints]
    batch_df['molecular_size'] = molecular_size
    collection_name.insert_many(batch_df.to_dict('records'))
    # collection_name.insert_many(batch_df.to_dict('records'))

0it [00:01, ?it/s]


InsertManyResult([ObjectId('6623771572b7fd24b8b7c8f1'), ObjectId('6623771572b7fd24b8b7c8f2'), ObjectId('6623771572b7fd24b8b7c8f3'), ObjectId('6623771572b7fd24b8b7c8f4'), ObjectId('6623771572b7fd24b8b7c8f5'), ObjectId('6623771572b7fd24b8b7c8f6'), ObjectId('6623771572b7fd24b8b7c8f7'), ObjectId('6623771572b7fd24b8b7c8f8'), ObjectId('6623771572b7fd24b8b7c8f9'), ObjectId('6623771572b7fd24b8b7c8fa'), ObjectId('6623771572b7fd24b8b7c8fb'), ObjectId('6623771572b7fd24b8b7c8fc'), ObjectId('6623771572b7fd24b8b7c8fd'), ObjectId('6623771572b7fd24b8b7c8fe'), ObjectId('6623771572b7fd24b8b7c8ff'), ObjectId('6623771572b7fd24b8b7c900'), ObjectId('6623771572b7fd24b8b7c901'), ObjectId('6623771572b7fd24b8b7c902'), ObjectId('6623771572b7fd24b8b7c903'), ObjectId('6623771572b7fd24b8b7c904'), ObjectId('6623771572b7fd24b8b7c905'), ObjectId('6623771572b7fd24b8b7c906'), ObjectId('6623771572b7fd24b8b7c907'), ObjectId('6623771572b7fd24b8b7c908'), ObjectId('6623771572b7fd24b8b7c909'), ObjectId('6623771572b7fd24b8b7c9